In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import re

model = LLM(
    "simplescaling/s1.1-32B",
    tensor_parallel_size=4,
)
tok = AutoTokenizer.from_pretrained("simplescaling/s1-32B")

stop_token_ids = tok("<|im_end|>")["input_ids"]

sampling_params = SamplingParams(
    max_tokens=32768,
    min_tokens=0,
    stop_token_ids=stop_token_ids,
)


INFO 02-17 20:31:06 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 02-17 20:31:11 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 02-17 20:31:11 config.py:1020] Defaulting to use mp for distributed inference
INFO 02-17 20:31:11 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='simplescaling/s1.1-32B', speculative_config=None, tokenizer='simplescaling/s1.1-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Observability

Loading safetensors checkpoint shards:   0% Completed | 0/29 [00:00<?, ?it/s]


(VllmWorkerProcess pid=1623466) INFO 02-17 20:32:52 model_runner.py:1077] Loading model weights took 15.3916 GB
(VllmWorkerProcess pid=1623467) INFO 02-17 20:32:53 model_runner.py:1077] Loading model weights took 15.3916 GB
INFO 02-17 20:32:53 model_runner.py:1077] Loading model weights took 15.3916 GB
(VllmWorkerProcess pid=1623465) INFO 02-17 20:32:54 model_runner.py:1077] Loading model weights took 15.3916 GB
(VllmWorkerProcess pid=1623466) (VllmWorkerProcess pid=1623467) INFO 02-17 20:33:03 worker.py:232] Memory profiling results: total_gpu_memory=47.43GiB initial_memory_usage=16.01GiB peak_torch_memory=17.92GiB memory_usage_post_profile=16.17GiB non_torch_memory=0.77GiB kv_cache_size=24.01GiB gpu_memory_utilization=0.90
(VllmWorkerProcess pid=1623465) INFO 02-17 20:33:03 worker.py:232] Memory profiling results: total_gpu_memory=47.43GiB initial_memory_usage=15.98GiB peak_torch_memory=17.92GiB memory_usage_post_profile=16.09GiB non_torch_memory=0.69GiB kv_cache_size=24.08GiB gpu_me

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/ssdscratch/byuan48/math_eval/data/AIME25.jsonl")
correct = 0
for i in range(len(dataset['train'])):
    prompt = dataset['train']['problem'][i]
    answer = dataset['train']['answer'][i]
    prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"
    o = model.generate(prompt, sampling_params=sampling_params)
    pattern = r"\\boxed\{(\d+)\}"
    matches = re.findall(pattern, o[0].outputs[0].text)
    print(f'matches: {matches}')
    if len(matches) == 0:
        print(f'no matches found')
    else:    
        if len(set(matches)) != 1:
            print(f'matches: {matches} not unique')
            print(f'current answer: {answer}')
        # remove the leading 0s
        matches[-1] = matches[-1].lstrip('0')
        if matches[-1] == str(answer):
            correct += 1
    print(f'correct: {correct}, in {i+1} questions')

print(f'correct: {correct}')

Processed prompts: 100%|██████████| 1/1 [02:24<00:00, 144.90s/it, est. speed input: 0.39 toks/s, output: 19.59 toks/s]


matches: ['70', '70']
correct: 1, in 1 questions


Processed prompts: 100%|██████████| 1/1 [17:02<00:00, 1022.83s/it, est. speed input: 0.18 toks/s, output: 18.60 toks/s]


matches: ['888', '888']
correct: 1, in 2 questions


Processed prompts: 100%|██████████| 1/1 [03:56<00:00, 236.64s/it, est. speed input: 0.60 toks/s, output: 19.49 toks/s]


...

Processed prompts: 100%|██████████| 1/1 [21:58<00:00, 1318.75s/it, est. speed input: 0.11 toks/s, output: 18.38 toks/s]


matches: ['503', '503']
correct: 12, in 28 questions


Processed prompts: 100%|██████████| 1/1 [12:08<00:00, 728.35s/it, est. speed input: 0.16 toks/s, output: 18.90 toks/s]


matches: ['264', '264']
correct: 12, in 29 questions


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [2]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/ssdscratch/byuan48/math_eval/data/AIME25.jsonl")
correct = 0
for i in range(29,len(dataset['train'])):
    prompt = dataset['train']['problem'][i]
    answer = dataset['train']['answer'][i]
    prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"
    o = model.generate(prompt, sampling_params=sampling_params)
    pattern = r"\\boxed\{(\d+)\}"
    matches = re.findall(pattern, o[0].outputs[0].text)
    print(f'matches: {matches}')
    if len(matches) == 0:
        print(f'no matches found')
    else:    
        if len(set(matches)) != 1:
            print(f'matches: {matches} not unique')
            print(f'current answer: {answer}')
        # remove the leading 0s
        matches[-1] = matches[-1].lstrip('0')
        if matches[-1] == str(answer):
            correct += 1
    print(f'correct: {correct}, in {i+1} questions')

print(f'correct: {correct}')

Processed prompts: 100%|██████████| 1/1 [15:13<00:00, 913.00s/it, est. speed input: 0.11 toks/s, output: 27.79 toks/s]

matches: ['188', '188']
correct: 0, in 30 questions
correct: 0


Results for o1-preview:
1.
